<a href="https://colab.research.google.com/github/BoBroccoli/ML/blob/main/c2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()
housing.head()
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
strat_train_set, strat_test_set = train_test_split(
    housing, test_size=0.2, stratify=housing["income_cat"], random_state=42)
strat_test_set["income_cat"].value_counts() / len(strat_test_set)
for set_ in (strat_train_set, strat_test_set):
  set_.drop("income_cat", axis=1, inplace=True)

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler()
)

def ratio_name(function_transformer, feature_names_in):
  return ["ratio"]

def calculate_ratio(X):
  return X[:, [0]] / X[:, [1]]

ratio_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(calculate_ratio, feature_names_out=ratio_name),
    StandardScaler()
)

housing_cat = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder()
)

class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, n_init=10,
                              random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

default_num_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

geo_cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1, random_state=42)

preprocessing = ColumnTransformer([
    ("bedrooms", ratio_pipeline, ["total_bedrooms", "total_rooms"]),
    ("rooms_per_house", ratio_pipeline, ["total_rooms", "households"]),
    ("people_per_house", ratio_pipeline, ["population", "households"]),
    ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                            "households", "median_income"]),
    ("geo", geo_cluster_simil, ["latitude", "longitude"]),
    ("cat", housing_cat, ["ocean_proximity"])
]
,remainder=default_num_pipeline
)
housing_prepared = preprocessing.fit_transform(housing)
housing_prepared_tr = pd.DataFrame(housing_prepared, columns=preprocessing.get_feature_names_out(), index=housing.index)
print(housing_prepared_tr.head())

       bedrooms__ratio  rooms_per_house__ratio  people_per_house__ratio  \
13096         1.846624               -0.866027                -0.330204   
14973        -0.508121                0.024550                -0.253616   
3785         -0.202155               -0.041193                -0.051041   
14689        -0.149006               -0.034858                -0.141475   
20507         0.963208               -0.666554                -0.306148   

       log__total_bedrooms  log__total_rooms  log__population  \
13096             1.324114          0.637892         0.456906   
14973            -0.252671         -0.063576        -0.711654   
3785             -0.925266         -0.859927        -0.941997   
14689             0.952773          0.943475         0.670700   
20507             1.437622          1.003590         0.719093   

       log__households  log__median_income  geo__Cluster 0 similarity  \
13096         1.310369           -1.071522               8.708837e-08   
14973       

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = make_pipeline(
    preprocessing,
    LinearRegression()
)
lin_reg.fit(housing, housing_labels)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1651: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x7db...
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('geo',
                                                  ClusterSimilarity(gamma=1,
                                                                    random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['ocean_proximity'])])),
                ('linearregression', LinearRegression())])

In [ ]:
housing_predictions = lin_reg.predict(housing)
print(housing_predictions[:5].round(-2))
print(housing_labels[:5].values)

[242800. 375900. 127500.  99400. 324600.]
[458300. 483800. 101700.  96100. 361800.]


In [ ]:
from sklearn.metrics import mean_squared_error
lin_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions))
print(lin_rmse)

68647.95686706658


In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = make_pipeline(
    preprocessing,
    DecisionTreeRegressor(random_state=42)
)
tree_reg.fit(housing, housing_labels)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1651: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x7db...
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('geo',
                                                  ClusterSimilarity(gamma=1,
                                                                    random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['ocean_proximity'])])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [ ]:
housing_predictions = tree_reg.predict(housing)
tree_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions))
print(tree_rmse)

0.0


In [ ]:
from sklearn.model_selection import cross_val_score
tree_rmse = -cross_val_score(tree_reg, housing, housing_labels,
                         scoring="neg_root_mean_squared_error", cv=10)

In [ ]:
pd.Series(tree_rmse).describe()

,0
count,10.000000
mean,66366.983603
std,1976.844743
min,63557.655007
25%,65004.623899
50%,65886.897085
75%,68129.026040
max,69530.301101


In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = make_pipeline(
    preprocessing,
    RandomForestRegressor(random_state=42)
)
forest_rmse = -cross_val_score(forest_reg, housing, housing_labels,
                         scoring="neg_root_mean_squared_error", cv=10)

In [ ]:
pd.Series(forest_rmse).describe()

,0
count,10.000000
mean,46938.209246
std,1018.397196
min,45522.649195
25%,46291.334639
50%,47021.703303
75%,47321.521991
max,49140.832210


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("randomForest", RandomForestRegressor(random_state=42))
])
param_grid = [
    {"preprocessing__geo__n_clusters": [5, 8, 10],
    "randomForest__max_features": [4, 6, 8]},
    {"preprocessing__geo__n_clusters": [10, 15],
     "randomForest__max_features": [6, 8, 10]}

]
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,
                         scoring="neg_root_mean_squared_error")
                        #  , refit=True)
grid_search.fit(housing, housing_labels)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1651: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('bedrooms',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('functiontransformer',
                                                                                          FunctionTransformer(feature_names_out=<f...
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder())]),
                                                                         ['ocean_proximity'])])),
                                       ('randomForest',
                                        RandomForestRegressor(random_state=42))]),
             param_grid=[{'preprocessing__geo__n_clusters': [5, 8, 10],
                          'randomForest__max_features': [4, 6, 8]},
                         {'preprocessing__geo__n_clusters': [10, 15],
                          'randomForest__max_features': [6, 8, 10]}],
             scoring='neg_root_mean_squared_error')

In [ ]:
grid_search.best_params_

{'preprocessing__geo__n_clusters': 15, 'randomForest__max_features': 6}

In [ ]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res[["mean_test_score"]] = -cv_res[["mean_test_score"]].round()
cv_res.sort_values(by="mean_test_score", ascending=True, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessing__geo__n_clusters,param_randomForest__max_features,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
12,8.872317,0.261005,0.202734,0.021056,15,6,"{'preprocessing__geo__n_clusters': 15, 'random...",-43006.650208,-43683.244212,-44158.332131,43616.0,472.565014,1
13,11.395409,0.229738,0.198404,0.016057,15,8,"{'preprocessing__geo__n_clusters': 15, 'random...",-43696.807883,-44011.195266,-44819.026722,44176.0,472.676329,2
7,8.466539,0.278087,0.205939,0.022424,10,6,"{'preprocessing__geo__n_clusters': 10, 'random...",-43709.661050,-44132.827198,-45226.940308,44356.0,639.295570,3
9,8.592977,0.186080,0.201965,0.024320,10,6,"{'preprocessing__geo__n_clusters': 10, 'random...",-43709.661050,-44132.827198,-45226.940308,44356.0,639.295570,3
6,6.160489,0.413204,0.187151,0.004731,10,4,"{'preprocessing__geo__n_clusters': 10, 'random...",-43797.854175,-44232.653866,-45100.371162,44377.0,541.452222,5


In [ ]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {'preprocessing__geo__n_clusters': randint(low=3, high=50),
                  'randomForest__max_features': randint(low=2, high=20)}
rnd_search = RandomizedSearchCV(full_pipeline, param_distributions=param_distribs, scoring="neg_root_mean_squared_error", cv=3, n_iter=10)
rnd_search.fit(housing, housing_labels)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1651: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                           SimpleImputer(strategy='median')),
                                                                                          ('standardscaler',
                                                                                           StandardScaler())]),
                                                                transformers=[('bedrooms',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('functiontransformer',
                                                                                                FunctionTransformer(feature_names_...
                                                                               ['ocean_proximity'])])),
                                             ('randomForest',
                                              RandomForestRegressor(random_state=42))]),
                   param_distributions={'preprocessing__geo__n_clusters': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7db8302ff310>,
                                        'randomForest__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7db8301f79d0>},
                   scoring='neg_root_mean_squared_error')

In [ ]:
rand_cv_res = pd.DataFrame(rnd_search.cv_results_)
rand_cv_res[["mean_test_score"]] = -rand_cv_res[["mean_test_score"]].round()
rand_cv_res.sort_values(by="mean_test_score", ascending=True, inplace=True)
rand_cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessing__geo__n_clusters,param_randomForest__max_features,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,20.204602,0.640449,0.214700,0.007691,47,13,"{'preprocessing__geo__n_clusters': 47, 'random...",-41585.936851,-42548.169943,-42988.263393,42374.0,585.576117,1
1,21.368229,0.566825,0.260640,0.039008,44,14,"{'preprocessing__geo__n_clusters': 44, 'random...",-41625.222027,-42507.363510,-43226.447447,42453.0,654.826368,2
9,8.514754,0.461195,0.211064,0.003081,37,5,"{'preprocessing__geo__n_clusters': 37, 'random...",-41175.570543,-42841.246482,-43534.941863,42517.0,990.079831,3
7,22.377034,1.432388,0.220883,0.010227,49,14,"{'preprocessing__geo__n_clusters': 49, 'random...",-41707.062880,-42681.373591,-43237.480627,42542.0,632.518325,4
3,12.225103,0.057292,0.203147,0.001910,26,8,"{'preprocessing__geo__n_clusters': 26, 'random...",-41847.550313,-42809.737363,-43313.951800,42657.0,608.309994,5


In [ ]:
final_model = rnd_search.best_estimator_
feature_importances = final_model["randomForest"].feature_importances_
feature_importances.round(2)
sorted(zip(feature_importances, preprocessing.get_feature_names_out()), reverse=True)

[(0.22436706000695145, 'log__median_income'),
 (0.06703726841293224, 'bedrooms__ratio'),
 (0.06102833240482823, 'people_per_house__ratio'),
 (0.05520135426732903, 'rooms_per_house__ratio'),
 (0.017084185928013272, 'geo__Cluster 6 similarity'),
 (0.015104267305320385, 'geo__Cluster 2 similarity'),
 (0.013872922863567007, 'geo__Cluster 8 similarity'),
 (0.010802028264455496, 'cat__ocean_proximity_NEAR OCEAN'),
 (0.009919991522895336, 'cat__ocean_proximity_ISLAND'),
 (0.008046279535572897, 'geo__Cluster 0 similarity'),
 (0.007096368393841968, 'cat__ocean_proximity_INLAND'),
 (0.0068081796678939885, 'cat__ocean_proximity_<1H OCEAN'),
 (0.006765350841786094, 'geo__Cluster 1 similarity'),
 (0.006751102756982313, 'log__total_rooms'),
 (0.00651083405389114, 'log__population'),
 (0.006469381803302829, 'geo__Cluster 5 similarity'),
 (0.006386292513880502, 'geo__Cluster 3 similarity'),
 (0.006212977714593877, 'log__total_bedrooms'),
 (0.006155073494395212, 'log__households'),
 (0.0059461498497963

In [ ]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()
final_prediction = final_model.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, final_prediction))
print(final_rmse)

42038.08665699851


In [ ]:
from scipy import stats
confidence = 0.95
squared_errors = (final_prediction - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

array([39906.48980345, 44066.69412527])

In [ ]:
import joblib

joblib.dump(final_model, "my_california_housing_model.pkl")

['my_california_housing_model.pkl']

In [ ]:
final_model_reloaded = joblib.load("my_california_housing_model.pkl")
sorted(zip(final_model_reloaded.predict(housing.iloc[:5]), strat_train_set['median_house_value'].iloc[:5]))

[(99600.0, 96100.0),
 (107908.0, 101700.0),
 (340366.04, 361800.0),
 (431395.09, 458300.0),
 (459888.09, 483800.0)]